In [250]:
import graphene
import requests
import pandas as pd

API_KEY = 'b105a60a-d678-4749-916a-b56e7a24d0d5'

In [251]:
'''
#"price", 
          "active_validators", 
          "annualized_rewards_usd", 
          "net_staking_flow_7d", 
          "staked_tokens", 
          "staking_marketcap", 

'''

'\n#"price", \n          "active_validators", \n          "annualized_rewards_usd", \n          "net_staking_flow_7d", \n          "staked_tokens", \n          "staking_marketcap", \n\n'

In [252]:
import requests
import pandas as pd

date = "2021-01-01"

endpoint = "https://api.stakingrewards.com/public/query"
query = f"""
query getStakingData($timeStart: Date, $symbol: [String!]) {{
  assets(where: {{symbols: $symbol}}, limit: 5){{
    symbol
    metrics(limit:50, 
      where:{{
        metricKeys: ["staking_ratio", "total_staking_wallets"], 
        createdAt_gt: $timeStart, 
        createdAt_lt: "2023-09-07"
      }}, 
      interval: hour,
      pickItem: last
    ) {{
      metricKey
      defaultValue
      createdAt
    }}
  }}
}}
"""

headers = {
  "Content-Type": "application/json",
  "X-API-KEY": API_KEY,
}

symbol = ["SOL", "ETH"]
time_start = "2022-01-01"

data = {"query": query,
        "variables": {
            "time_start": time_start,
            "symbol": symbol
        }}

response = requests.post(endpoint, json=data, headers=headers)

print(response)
if response.status_code == 200:
    result = response.json()
    print(result)
    # Assume the JSON structure is: { "data": { "assets": [ { ... }, ... ] } }
    assets = result.get("data", {}).get("assets", [])
    # Flattening the main asset information
    df_assets = pd.json_normalize(assets)
    print("Assets DataFrame:")
    print(df_assets.head())
    
    # Flattening the nested "metrics" field, including asset-level metadata (symbol)
    df_metrics = pd.json_normalize(assets, record_path='metrics', meta=['symbol'])
    print("\nMetrics DataFrame:")
    print(df_metrics.head())
else:
    print("Error occurred:", response.status_code)


<Response [200]>
{'data': {'assets': [{'symbol': 'ETH', 'metrics': [{'metricKey': 'staking_ratio', 'defaultValue': 0.5930814090680153, 'createdAt': '2020-12-01T01:40:21Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930814090680153, 'createdAt': '2020-12-01T02:00:00Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930754618203001, 'createdAt': '2020-12-01T03:40:18Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930754618203001, 'createdAt': '2020-12-01T04:00:00Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930693294371676, 'createdAt': '2020-12-01T05:40:17Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930693294371676, 'createdAt': '2020-12-01T06:00:00Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930637261897744, 'createdAt': '2020-12-01T07:40:17Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930637261897744, 'createdAt': '2020-12-01T08:00:00Z'}, {'metricKey': 'staking_ratio', 'defaultValue': 0.5930581282665011, 'createdAt': '2020-12-01T

In [255]:
display(df_metrics)

,metricKey,defaultValue,createdAt,symbol
0,staking_ratio,0.593081,2020-12-01T01:40:21Z,ETH
1,staking_ratio,0.593081,2020-12-01T02:00:00Z,ETH
2,staking_ratio,0.593075,2020-12-01T03:40:18Z,ETH
3,staking_ratio,0.593075,2020-12-01T04:00:00Z,ETH
4,staking_ratio,0.593069,2020-12-01T05:40:17Z,ETH
...,...,...,...,...
95,staking_ratio,35.138626,2021-02-19T06:00:00Z,SOL
96,staking_ratio,35.138627,2021-02-19T07:40:47Z,SOL
97,staking_ratio,35.138627,2021-02-19T08:00:00Z,SOL
98,staking_ratio,35.138627,2021-02-19T09:40:33Z,SOL
